In [ ]:
import IPython; IPython.Application.instance().kernel.do_shutdown(True)

In [1]:
!pip install --no-cache-dir xgboost

In [2]:
!pip install --upgrade sagemaker boto3

  Using cached sagemaker-3.4.1-py3-none-any.whl.metadata (20 kB)
  Using cached sagemaker_core-2.4.1-py3-none-any.whl.metadata (5.5 kB)
  Using cached rich-13.9.4-py3-none-any.whl.metadata (18 kB)
Using cached sagemaker-3.4.1-py3-none-any.whl (11 kB)
Using cached sagemaker_core-2.4.1-py3-none-any.whl (1.3 MB)
Using cached rich-13.9.4-py3-none-any.whl (242 kB)
  Attempting uninstall: rich
    Found existing installation: rich 14.3.3
    Uninstalling rich-14.3.3:
      Successfully uninstalled rich-14.3.3
  Attempting uninstall: sagemaker-core
    Found existing installation: sagemaker-core 1.0.77
    Uninstalling sagemaker-core-1.0.77:
      Successfully uninstalled sagemaker-core-1.0.77
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 2.257.0
    Uninstalling sagemaker-2.257.0:
      Successfully uninstalled sagemaker-2.257.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the sourc

In [3]:
%pip install pandas pyarrow s3fs

  Using cached aiobotocore-2.26.0-py3-none-any.whl.metadata (25 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 150.9 MB/s eta 0:00:00
  Attempting uninstall: botocore
    Found existing installation: botocore 1.42.54
    Uninstalling botocore-1.42.54:
      Successfully uninstalled botocore-1.42.54
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
amazon-sagemaker-sql-execution 0.1.9 requires snowflake-connector-python<4.0,>=3.8.1, which is not installed.
boto3 1.42.54 requires botocore<1.43.0,>=1.42.54, but you have botocore 1.41.5 which is incompatible.
sagemaker-mlops 1.4.1 requires botocore<2.0,>=1.42.2, but you have botocore 1.41.5 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd

s3_path = "s3://bucket2-curated-stock/curated/stock_market/parquet/stock_clean.parquet"

df = pd.read_parquet(s3_path)
print(df.columns)

df.head()

Index(['date', 'symbol', 'open', 'high', 'low', 'close', 'volume', 'range',
       'daily_return_pct'],
      dtype='object')


,date,symbol,open,high,low,close,volume,range,daily_return_pct
0,2024-01-01,AAPL,154.72,161.22,151.89,159.55,3494467,9.33,3.121768
1,2024-01-03,AAPL,142.91,144.99,138.40,141.21,10960425,6.59,-1.189560
2,2024-01-04,AAPL,183.98,189.54,182.29,186.80,6915432,7.25,1.532775
3,2024-01-14,AAPL,357.31,361.14,349.48,350.06,7535315,11.66,-2.029050
4,2024-01-15,AAPL,102.33,108.73,98.64,107.00,11891970,10.09,4.563667


In [4]:
import sys
print(sys.version)
print(sys.executable)

3.11.11 | packaged by conda-forge | (main, Dec  5 2024, 14:17:24) [GCC 13.3.0]
/opt/conda/bin/python


In [5]:
df = df.dropna()

X = df.drop(columns=["close"])  # target
y = df["close"]

split = int(len(df) * 0.8)

X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

In [6]:
import pandas as pd

train_df = X_train.copy()
train_df["close"] = y_train

test_df = X_test.copy()
test_df["close"] = y_test

In [7]:
import pandas as pd
from xgboost import XGBRegressor

# 1) Prepare features: drop 'date' and cast object to category
def prep_minimal(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    # Drop datetime column to avoid dtype error
    if 'date' in df.columns:
        df = df.drop(columns=['date'])
    # Cast object columns to pandas 'category' for XGBoost
    obj_cols = df.select_dtypes(include=['object']).columns
    for c in obj_cols:
        df[c] = df[c].astype('category')
    return df

X_train_pre = prep_minimal(X_train)
X_test_pre  = prep_minimal(X_test)

# Ensure y is numeric
y_train = pd.to_numeric(y_train, errors='coerce')
y_test  = pd.to_numeric(y_test, errors='coerce')

# 2) Train with categorical support enabled
model = XGBRegressor(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.9,
    colsample_bytree=0.9,
    random_state=42,
    tree_method="hist",
    enable_categorical=True,   # <-- important
)

model.fit(X_train_pre, y_train)
print("Model training complete ✅")

Model training complete ✅


In [8]:
import pandas as pd
from pandas.api.types import is_datetime64_any_dtype as is_datetime

def prep_with_date_features(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    # 1) Handle datetime -> numeric features, then drop original
    if 'date' in df.columns and is_datetime(df['date']):
        df['year']  = df['date'].dt.year.astype('int16')
        df['month'] = df['date'].dt.month.astype('int8')
        df['day']   = df['date'].dt.day.astype('int8')
        df['dow']   = df['date'].dt.dayofweek.astype('int8')
        df = df.drop(columns=['date'])

    # 2) Cast object columns to 'category'
    obj_cols = df.select_dtypes(include=['object']).columns.tolist()
    for c in obj_cols:
        df[c] = df[c].astype('category')

    # 3) Optional: fill NaNs
    for col in df.columns:
        if pd.api.types.is_numeric_dtype(df[col]):
            df[col] = df[col].fillna(0)
        elif pd.api.types.is_categorical_dtype(df[col]):
            df[col] = df[col].cat.add_categories(["__MISSING__"]).fillna("__MISSING__")
    return df

In [9]:
# Build train features
X_train_pre = prep_with_date_features(X_train)

# Keep the training categories to reapply on test
cat_cols = X_train_pre.select_dtypes(include="category").columns.tolist()
train_cats = {c: X_train_pre[c].cat.categories for c in cat_cols}

from xgboost import XGBRegressor

model = XGBRegressor(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.9,
    colsample_bytree=0.9,
    random_state=42,
    tree_method="hist",
    enable_categorical=True,   # critical for categorical 'category' dtype
)
model.fit(X_train_pre, y_train)

,objective,'reg:squarederror'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,0.9
,device,None
,early_stopping_rounds,None
,enable_categorical,True
,eval_metric,None


In [10]:
# Apply same preprocessing
X_test_pre = prep_with_date_features(X_test)

# Align test categorical columns to training categories (order & set)
for c in cat_cols:
    if c in X_test_pre.columns:
        X_test_pre[c] = X_test_pre[c].astype('category')
        X_test_pre[c] = X_test_pre[c].cat.set_categories(train_cats[c])

# Predict using preprocessed features
preds = model.predict(X_test_pre)

from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

rmse = np.sqrt(mean_squared_error(y_test, preds))
r2 = r2_score(y_test, preds)

print("RMSE:", rmse)
print("R2 Score:", r2)

RMSE: 1.7031788935870933
R2 Score: 0.999598440644214


In [11]:
print("Train dtypes:\n", X_train_pre.dtypes)
print("Test dtypes:\n", X_test_pre.dtypes)

# Should show only numeric and category
bad_train = [c for c, dt in X_train_pre.dtypes.items() if dt.kind not in ['i', 'u', 'f', 'b'] and 'category' not in str(dt)]
bad_test  = [c for c, dt in X_test_pre.dtypes.items()  if dt.kind not in ['i', 'u', 'f', 'b'] and 'category' not in str(dt)]
print("Bad train cols:", bad_train)
print("Bad test cols:", bad_test)


Train dtypes:
 symbol              category
open                 float64
high                 float64
low                  float64
volume                 int64
range                float64
daily_return_pct     float64
year                   int16
month                   int8
day                     int8
dow                     int8
dtype: object
Test dtypes:
 symbol              category
open                 float64
high                 float64
low                  float64
volume                 int64
range                float64
daily_return_pct     float64
year                   int16
month                   int8
day                     int8
dow                     int8
dtype: object
Bad train cols: []
Bad test cols: []


In [12]:
import pickle
bundle = {
    "model": model,
    "cat_cols": X_train_pre.select_dtypes(include="category").columns.tolist(),
    "train_categories": {c: X_train_pre[c].cat.categories for c in X_train_pre.select_dtypes(include="category").columns},
    # Optionally include the function source or a callable for preprocessing
}
with open("xgb_model_bundle.pkl", "wb") as f:
    pickle.dump(bundle, f)

In [13]:
import boto3
s3 = boto3.client("s3")
s3.upload_file("xgb_model_bundle.pkl", "bucket2-curated-stock", "ml-artifacts/stock-xgb/model/xgb_model_bundle.pkl")

In [14]:
%%bash
cat > inference.py << 'EOF'
import os
import io
import json
import pickle
import pandas as pd
from pandas.api.types import is_datetime64_any_dtype as is_datetime

def prep_with_date_features(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    # Parse date if string
    if "date" in df.columns and df["date"].dtype == object:
        df["date"] = pd.to_datetime(df["date"], errors="coerce")

    # Extract features then drop
    if "date" in df.columns and is_datetime(df["date"]):
        df["year"] = df["date"].dt.year.astype("int16")
        df["month"] = df["date"].dt.month.astype("int8")
        df["day"] = df["date"].dt.day.astype("int8")
        df["dow"] = df["date"].dt.dayofweek.astype("int8")
        df = df.drop(columns=["date"])

    # Cast objects → category
    for c in df.select_dtypes(include=["object"]).columns:
        df[c] = df[c].astype("category")

    # Fill NA
    for col in df.columns:
        if pd.api.types.is_numeric_dtype(df[col]):
            df[col] = df[col].fillna(0)
        else:
            df[col] = df[col].astype("category").cat.add_categories(["__MISSING__"]).fillna("__MISSING__")

    return df

def model_fn(model_dir):
    path = os.path.join(model_dir, "xgb_model_bundle.pkl")
    with open(path, "rb") as f:
        bundle = pickle.load(f)
    return bundle

def input_fn(request_body, content_type):
    if content_type == "application/json":
        data = json.loads(request_body)
        if isinstance(data, dict) and "instances" in data:
            df = pd.DataFrame(data["instances"])
        else:
            df = pd.DataFrame(data)
    else:
        df = pd.read_csv(io.StringIO(request_body))
    return df

def predict_fn(df, model_bundle):
    model = model_bundle["model"]
    cat_cols = model_bundle["cat_cols"]
    train_cats = model_bundle["train_categories"]

    X = prep_with_date_features(df)

    # align categories
    for c in cat_cols:
        if c in X.columns:
            X[c] = X[c].astype("category")
            allowed = list(train_cats[c]) + ["__UNSEEN__"]
            X[c] = X[c].where(X[c].isin(train_cats[c]), "__UNSEEN__")
            X[c] = X[c].cat.set_categories(allowed)

    preds = model.predict(X)
    return preds

def output_fn(predictions, accept):
    if accept == "application/json":
        return json.dumps({"predictions": predictions.tolist()})
    return "\n".join(map(str, predictions))
EOF

In [15]:
import os, tarfile, boto3

bucket = "bucket2-curated-stock"
pickle_key = "ml-artifacts/stock-xgb/model/xgb_model_bundle.pkl"
model_tar_key = "ml-artifacts/stock-xgb/model/model.tar.gz"

# Prepare folder
os.makedirs("model_artifacts", exist_ok=True)

# Download pickle
s3 = boto3.client("s3")
s3.download_file(bucket, pickle_key, "model_artifacts/xgb_model_bundle.pkl")

# Copy inference.py & requirements.txt
os.system("cp inference.py model_artifacts/")
os.system("cp requirements.txt model_artifacts/")

# Create tar.gz
with tarfile.open("model.tar.gz", "w:gz") as tar:
    tar.add("model_artifacts", arcname=".")

# Upload
s3.upload_file("model.tar.gz", bucket, model_tar_key)
print("Uploaded:", f"s3://{bucket}/{model_tar_key}")


Uploaded: s3://bucket2-curated-stock/ml-artifacts/stock-xgb/model/model.tar.gz


In [16]:
pip install sagemaker==2.*

Note: you may need to restart the kernel to use updated packages.


## Model Serving

Deploy the trained XGBoost model as a SageMaker endpoint for real-time inference.

In [20]:
%%bash
cat > requirements.txt << 'EOF'
xgboost==2.1.4
pandas>=2.3.0
numpy>=1.26.4
scikit-learn>=1.7.2
EOF

In [57]:
endpoint_name = "stock-xgb-endpoint-new"

try:
    sm.delete_endpoint(EndpointName=endpoint_name)
    print(f"Deleted existing endpoint: {endpoint_name}")
    # Wait for deletion
    import time
    time.sleep(30)
except:
    pass

response = sm.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name
)

print(f"Endpoint creation started: {endpoint_name}")

Endpoint creation started: stock-xgb-endpoint-new


In [22]:
pip install -U sagemaker

  Using cached sagemaker-3.4.1-py3-none-any.whl.metadata (20 kB)
  Using cached sagemaker_core-2.4.1-py3-none-any.whl.metadata (5.5 kB)
  Using cached sagemaker_train-1.4.1-py3-none-any.whl.metadata (7.8 kB)
  Using cached sagemaker_serve-1.4.1-py3-none-any.whl.metadata (1.6 kB)
  Using cached sagemaker_mlops-1.4.1-py3-none-any.whl.metadata (5.7 kB)
  Using cached rich-13.9.4-py3-none-any.whl.metadata (18 kB)
  Using cached deepdiff-8.6.1-py3-none-any.whl.metadata (8.6 kB)
  Using cached sagemaker_schema_inference_artifacts-0.0.5-py3-none-any.whl.metadata (2.3 kB)
  Using cached tritonclient-2.65.0-py3-none-manylinux1_x86_64.whl.metadata (3.0 kB)
  Using cached onnx-1.20.1-cp311-cp311-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (8.4 kB)
  Using cached mlflow-3.10.0-py3-none-any.whl.metadata (31 kB)
  Using cached s3transfer-0.16.0-py3-none-any.whl.metadata (1.7 kB)
  Using cached mlflow_skinny-3.10.0-py3-none-any.whl.metadata (32 kB)
  Using cached mlflow_tracing-3.10.0-py

In [23]:
image_uri = "763104351884.dkr.ecr.us-east-2.amazonaws.com/pytorch-inference:2.1.0-cpu-py310"
print("Image:", image_uri)

Image: 763104351884.dkr.ecr.us-east-2.amazonaws.com/pytorch-inference:2.1.0-cpu-py310


In [25]:
import boto3

sm = boto3.client("sagemaker")

model_name = "stock-xgb-model-final1"

container = {
    "Image": image_uri,
    "ModelDataUrl": "s3://bucket2-curated-stock/ml-artifacts/stock-xgb/model/model.tar.gz",
    "Environment": {
        "SAGEMAKER_PROGRAM": "inference.py",
        "SAGEMAKER_SUBMIT_DIRECTORY": "/opt/ml/model",
        "SAGEMAKER_REQUIREMENTS": "requirements.txt"
    }
}

sm.create_model(
    ModelName=model_name,
    ExecutionRoleArn="arn:aws:iam::880192676325:role/service-role/AmazonSageMakerAdminIAMExecutionRole",
    PrimaryContainer=container
)

print("MODEL CREATED:", model_name)

MODEL CREATED: stock-xgb-model-final1


In [26]:
endpoint_config_name = "stock-xgb-config-final1"

sm.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "VariantName": "AllTraffic",
            "ModelName": model_name,
            "InitialInstanceCount": 1,
            "InstanceType": "ml.m5.large",
            "InitialVariantWeight": 1
        }
    ]
)

print("CONFIG CREATED:", endpoint_config_name)

CONFIG CREATED: stock-xgb-config-final1


In [27]:
endpoint_name = "stock-xgb-endpoint-final1"

sm.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name
)

print("DEPLOY STARTED:", endpoint_name)

DEPLOY STARTED: stock-xgb-endpoint-final1


In [28]:
import time

while True:
    resp = sm.describe_endpoint(EndpointName=endpoint_name)
    status = resp["EndpointStatus"]
    print("STATUS:", status)
    if status in ["InService", "Failed"]:
        print("FINAL:", resp.get("FailureReason"))
        break
    time.sleep(20)

STATUS: Creating
STATUS: Creating
STATUS: Creating
STATUS: Creating
STATUS: Creating
STATUS: Creating
STATUS: Creating
STATUS: Creating
STATUS: Creating
STATUS: Creating
STATUS: InService
FINAL: None


In [ ]:
import boto3

sm = boto3.client("sagemaker")

resp = sm.describe_endpoint(EndpointName="stock-xgb-endpoint-final1")
print(resp["EndpointStatus"])

In [ ]:
import boto3
import json

runtime = boto3.client("sagemaker-runtime")

payload = json.dumps(sample)

response = runtime.invoke_endpoint(
    EndpointName="stock-xgb-endpoint-final",
    ContentType="application/json",
    Body=payload
)

print(response["Body"].read().decode())

In [80]:
import boto3

region = "us-east-2"  # <-- your deploy region
sm = boto3.client("sagemaker", region_name=region)

desc = sm.describe_endpoint(EndpointName="stock-xgb-endpoint-final")
print("Status:", desc["EndpointStatus"])
print("FailureReason:", desc.get("FailureReason"))

Status: InService
FailureReason: None


In [30]:
import boto3, json
from botocore.config import Config

region = "us-east-2"
endpoint_name = "stock-xgb-endpoint-final"

runtime = boto3.client(
    "sagemaker-runtime",
    region_name=region,
    config=Config(
        connect_timeout=10,
        read_timeout=300,                      # <-- override the default 60s
        retries={"max_attempts": 8, "mode": "standard"}
    ),
)

print("Client read_timeout:", runtime.meta.config.read_timeout)  # should print 300

Client read_timeout: 300


In [31]:
test_record = {
    "date": "2024-01-01",
    "symbol": "AAPL",
    "open": 154.72,
    "high": 161.22,
    "low": 151.89,
    "volume": 3_494_467,
    "range": 9.33,
    "daily_return_pct": 3.12,
}

resp = runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType="application/json",     # your input_fn reads JSON
    Accept="application/json",          # your output_fn returns JSON when Accept=application/json
    Body=json.dumps({"instances": [test_record]}),
)
result = json.loads(resp["Body"].read().decode("utf-8"))
print("Prediction:", result["predictions"][0])

ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received server error (0) from primary with message "Your invocation timed out while waiting for a response from container primary. Review the latency metrics for each container in Amazon CloudWatch, resolve the issue, and try again.". See https://us-east-2.console.aws.amazon.com/cloudwatch/home?region=us-east-2#logEventViewer:group=/aws/sagemaker/Endpoints/stock-xgb-endpoint-final in account 880192676325 for more information.